In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plot
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('data/ems_weather.csv')

### **Model 1: Kitchen Sink**

In [7]:
# NOTE: features[N:M] actually is features[N] through features [M-1] -- it's non-inclusive

# features[0]       --- 'CAD_INCIDENT_ID'                                            ---- DROP (only an index)
# features[1]       --- 'INITIAL_SEVERITY_LEVEL_CODE'
# features[2]       --- 'FINAL_SEVERITY_LEVEL_CODE'
# features[3]       --- 'FIRST_ASSIGNMENT_DATETIME'                                  ---- DROP
# features[4]       --- 'VALID_DISPATCH_RSPNS_TIME_INDC' 
# features[5]       --- 'DISPATCH_RESPONSE_SECONDS_QY'                               ---- DROP
# features[6]       --- 'FIRST_ACTIVATION_DATETIME' 
# features[7]       --- 'VALID_INCIDENT_RSPNS_TIME_INDC'
# features[8]       --- 'INCIDENT_RESPONSE_SECONDS_QY'                               ------> y
# features[9]       --- 'INCIDENT_TRAVEL_TM_SECONDS_QY'
# features[10]      --- 'INCIDENT_CLOSE_DATETIME' 
# features[11]      --- 'HELD_INDICATOR' 
# features[12]      --- 'REOPEN_INDICATOR'
# features[13]      --- 'SPECIAL_EVENT_INDICATOR' 
# features[14]      --- 'STANDBY_INDICATOR'
# features[15]      --- 'TRANSFER_INDICATOR'
# features[16:96]   --- 'INITIAL_CALL_TYPE' dummies
# features[96:186]  --- 'FINAL_CALL_TYPE' dummies
# features[186:195] --- 'INCIDENT_DISPOSITION_CODE' dummies
# features[195:198] --- 'BOROUGH' dummies
# features[198:230] --- 'INCIDENT_DISPATCH_AREA' dummies
# features[230:594] --- 'ZIPCODE' dummies
# features[594:670] --- 'POLICEPRECINCT' dummies
# features[670:720] --- 'CITYCOUNCILDISTRICT' dummies
# features[720:788] --- 'COMMUNITYDISTRICT' dummies
# features[788:819] --- 'COMMUNITYSCHOOLDISTRICT' dummies
# features[829:831] --- 'CONGRESSIONALDISTRICT' dummies
# features[831]     --- 'CALL_YEAR'
# features[832]     --- 'CALL_DAY'
# features[833]     --- 'CALL_TIME'
# features[834]     --- 'CALL_UTC'
# features[835:846] --- 'CALL_MONTH' dummies
# features[846]     --- 'CALL_MONTH' (numeric)
# features[847]     --- 'AWIND'
# features[848]     --- 'PRCP'
# features[849]     --- 'SNOW'
# features[850]     --- 'SNWD'
# features[851]     --- 'TMAX'
# features[852]     --- 'TMIN'
# features[853]     --- 'TAVG_CALC'

In [22]:
features = df.columns

# definitely drop these:
drop_features = ['INCIDENT_RESPONSE_SECONDS_QY', 
                 'CAD_INCIDENT_ID', 
                 'FIRST_ASSIGNMENT_DATETIME', 
                 'DISPATCH_RESPONSE_SECONDS_QY']

# suggest dropping these: 
drop_features += list(features[230:594]) # zip codes
drop_features += list(features[670:831]) # city council district, community district, community school dist, congressional district


X = df.drop(columns=drop_features)

y = df['INCIDENT_RESPONSE_SECONDS_QY']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [23]:
linreg = LinearRegression()

linreg.fit(X_train, y_train)

print(f'Training: {linreg.score(X_train, y_train)}')
print(f'Testing: {linreg.score(X_test, y_test)}')

Training: 0.9960344887121069
Testing: -0.3385918115893678


This does not seem likely, not sure yet why this is the result but there is definitely something not quite right here.

In [24]:
lasso = Lasso(max_iter = 10_000)

lasso.fit(X_train, y_train)

print(f'Training: {lasso.score(X_train, y_train)}')
print(f'Testing: {lasso.score(X_test, y_test)}')

Training: 0.9958146286844293
Testing: -0.13915108084119465


In [25]:
ridge = Ridge()

ridge.fit(X_train, y_train)

print(f'Training: {lasso.score(X_train, y_train)}')
print(f'Testing: {lasso.score(X_test, y_test)}')

Training: 0.9958146286844293
Testing: -0.13915108084119465


C:\Users\tacia\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=1.16905e-21): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,


I realized the issue here.  Left in a lot of data that was used to create our new dependent variable.  This has been corrected more or less.